<a href="https://colab.research.google.com/github/GDdeCastro/Computational_Methods_in_Physics/blob/main/planar_ising_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def initial_config(N, M):
  spins_config0 = np.random.choice([-1,1], N*M) # gera um grande vetor de +- uns
  spins_config0 = np.reshape(spins_config0,(N,M)) # transforma o vetor numa matriz
  return spins_config0 # retorna uma configuração inicial de spins aleatória em 2D

In [ ]:
def calculate_energy(spins_config, J, h):
  N, M = np.shape(spins_config) # obtém as dimensões da matriz de spins
  sum = 0 
  for i in range(N):
    for j in range(M): # para cada posição na malha
      sum += -J*spins_config[i,j]*(spins_config[(i+1)%N,j]+spins_config[i,(j+1)%M]) # soma a contribuição dos vizinhos superiores
      sum += -h*spins_config[i,j] # adiciona a contribuição do campo magnético
  return sum

In [ ]:
def plot_config(spins_config, name):
  plt.figure(figsize=(8,8)) # cria figura
  plt.imshow(spins_config, 'Greys_r') # plota grade que mostra a configuração
  # de spins em 2D
  plt.tick_params(left = False, bottom = False, labelleft = False, labelbottom = False)
  plt.savefig(name+'.png') # salva a imagem

In [ ]:
def delta_e(s, i, j, J, h):
  '''
  Essa funcao calcula a variacao da energia (O Eflip)
  '''
  N, M = np.shape(s) # pego o tamanho da matriz
  s_cand = s[i,j] # Pego o spin candidato

  # Aqui calculo efetivamente o DE
  neigh = s[(i-1)%N,j] + s[(i+1)%N,j] + s[i,(j-1)%M] + s[i,(j+1)%M] # funciona inclusive nas bordas
  sum = -J*(2*s_cand*(neigh)) - 2*h*s_cand # computo DE
  # importante: A matriz que é recebida já deve ser a matriz com o spin flipado!
  # Em outras palavras, o delta_e é calculado supondo que recebe o valor de S_novo
  return sum

In [ ]:
#@jit(fastmath=True)
def sweep_config(spins_config, J, h, beta, E):
  '''
  Essa funcao varre todos os spins da malha de spins aleatoriamente
  e flipa de acordo com a possibilidade do algoritmo de metropolis
  '''
  E0 = E
  N, M = np.shape(spins_config) # Obtenho o tamanho da malha
  n = np.arange(N) # Crio uma lista de índices das linhas
  m = np.arange(M) # Crio uma lista de índices das colunas
  np.random.shuffle(n) # Embaralho a primeira lista
  np.random.shuffle(m) # Embaralho a segunda lista
  for nn in n: # percorro a malha usando os índices das listas embaralhadas
    for mm in m:
      spins_config[nn, mm] *= -1 # flipo o spin (previamente)
      DE = delta_e(spins_config, nn, mm, J, h) # calculo a variação de energia
      E += DE # atualizo a energia
      if DE > 0: # Se DE > 0, vejo a possibilidade de flipar mesmo assim
        p_flip = np.exp(-DE*beta) # Probabilidade de flipar
        r = np.random.uniform(0,1) # número aleatório
        if p_flip < r: # Probabilidade pequena
          spins_config[nn, mm] *= -1 # desflipo o spin
          E -= DE # retiro o incremento da energia
  return E # retorno a nova energia

In [ ]:
def run_simulation(Nmax, Ts, hs, J, k, N, M, inicio_aleatorio = True):

  '''
  Aqui, efetivamente, rodo a simulação
  '''

  EE = [] # lista para salvar os dados de energia para cada h
  C = [] # lista para salvar os dados de capacidade térmica
  MM = [] # lista para salvar os dados de magnetizacao para cada h
  CHI = [] # lista para salvar os dados de susceptibilidade magnetica para cada h

  for hh in range(np.size(hs)): # para cada h na lista hs
    Es = [] # lista para guardar as energias médias
    Es2 = [] # lista para guardar as energias medias quadradas
    Mags = [] # lista para guardar as magnetizacoes médias
    Mags2 = [] # lista para guardar as magnetizacoes médias quadradas
    Cs = [] # ista para guardar as capacidades térmicas
    Chis = [] # lista para guardar as susceptibilidades magnéticas
    if inicio_aleatorio:
      config0 = initial_config(N, M) # crio uma configuração aleatória de spins
    else:
      config0 = np.ones((N,M)) # crio uma configuração magnetizada (spins up)
    for t in range(np.size(Ts)):  # para cada temperatura na lista ts
      beta = 1/(k*Ts[t]) # calculo beta
      ms = [] # lista para guardar as magnetizacoes por sitio (em módulo)
      es = [] # lista para guardar as energias
      spins_config = np.copy(config0) 
      E = calculate_energy(spins_config, J, hs[hh]) # calculo a energia inicial
      m = np.absolute(np.mean(spins_config)) # calculo a magnetizacao inicial
      es.append(E) # salvo a primeira energia
      ms.append(m) # salvo a primeira magnet
      for cont in range(Nmax-1): # itero nas varreduras
        E = sweep_config(spins_config, J, hs[hh], beta, E) # varro e salvo a energia nova
        es.append(E) # salvo a energia
        ms.append(np.sum(spins_config)) # salvo a magnetizacao
      Mags.append(np.mean(ms)) # salvo a magnetizacao média total
      Mags2.append(np.mean(np.array(ms)**2)) # salvo a magnetização quadrada média
      Es.append(np.mean(es)) # salvo a energia média total
      Es2.append(np.mean(np.array(es)**2)) # Salvo a energia quadrada média
      Cs.append(-(Es[-1]**2 - Es2[-1])/(k**2*Ts[t]**2)) # Salvo o calor específico
      Chis.append(-(Mags[-1]**2 - Mags2[-1])/(k*Ts[t])) # Salvo a susceptibilidade magnética
    EE.append(Es) # Salvo os dados para cada h
    MM.append(Mags)
    C.append(Cs)
    CHI.append(Chis)
  Ts = np.array(Ts) # transformo todas as listas em arrays numpy 
  hs = np.array(hs)
  EE = np.array(EE)/(N*M)
  MM = np.array(np.absolute(MM))/(N*M)
  C = np.array(C)/(N*M)
  CHI = np.array(CHI)/(N*M)
  return Ts, hs, EE, MM, C, CHI

In [ ]:
def mag_onsager(k, J, T):
  '''
  Essa função calcula a magnetização prevista por Onsager
  para uma configuração de spins em 2D com h = 0
  '''
  beta = (k*T)**(-1) # calcula a constante beta
  if T <= 2*J/(k*np.log(1+np.sqrt(2))): # Se a temperatura é inferior à crítica
    return (1-(np.sinh(2*beta*J))**(-4))**(1/8) # magnetização espontânea de Onsager
  else: # A função é por partes
    return 0

In [ ]:
def plot_observables(Ts, hs, EE, MM, C, CHI, k, J, lit = True):

  '''
  Essa função plota os observáveis calculados. A variável lit serve para
  plotar ou não a função de magnetização esperada por Onsager sobre os dados computados 
  computacionalmente.
  '''

  fig, ax = plt.subplots(figsize=(8,8))

  tc = 2*J/(k*np.log(1+np.sqrt(2)))
  t_aux = np.arange(0.01,4,0.01)
  m_onsager = np.array([mag_onsager(k,J,t) for t in t_aux])

  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for hh in range(np.size(hs)):
      ax.plot(Ts, MM[hh], 's--', label=f'h = {hs[hh]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel('|<M(T)>|', fontsize=18)
      ax.set_title('Magnetização Média em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  if lit:
    ax.plot(t_aux, m_onsager, 'k-', label=f'Onsager Solution')
  ax.legend(fontsize=(14))
  plt.savefig(f'mag_vs_temp_hs')
  
  fig1, ax = plt.subplots(figsize=(8,8))
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for hh in range(np.size(hs)):
      ax.plot(Ts, EE[hh], 's--', label=f'h = {hs[hh]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel('<E(T)>', fontsize=18)
      ax.legend(fontsize=(14))
      #ax.set_xscale('log')
      #ax.set_yscale('log')
      ax.set_title('Energia Média por Sítio em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  plt.savefig(f'en_vs_temp_hs')

  fig2, ax = plt.subplots(figsize=(8,8))
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for hh in range(np.size(hs)):
      #ax.axvline(x = find_max_point(C[hh], Ts), color='red',linestyle='dashed', label=f'$T_c$ = {find_max_point(C[hh], Ts):.3f}')
      ax.plot(Ts, C[hh], 's--', label=f'h = {hs[hh]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel('C(T)', fontsize=18)
      ax.legend(fontsize=(14))
      ax.set_title('Calor Específico em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  ax.set_xlim(1, 2.8)
  ax.set_ylim(0, 3)
  plt.savefig(f'C_vs_temp_hs')

  fig3, ax = plt.subplots(figsize=(8,8))
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for hh in range(np.size(hs)):
      ax.plot(Ts, CHI[hh], 's--', label=f'h = {hs[hh]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel(r'$\chi$(T)', fontsize=18)
      ax.legend(fontsize=(14))
      ax.set_title('Susceptibilidade Magnética em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  ax.set_xlim(1.4, 2.8)
  plt.savefig(f'CHI_vs_temp_hs')


In [ ]:
Ts, hs, EE, MM, C, CHI = run_simulation(1000, np.arange(0.1,15,0.2), [0,1,2,3,4], 1, 1, 20, 20, False)

KeyboardInterrupt: ignored

In [ ]:
plot_observables(Ts, hs, EE, MM, C, CHI, 1, 1, False)

In [ ]:
def derivada(y, x):
  '''
  Essa função calcula uma derivada numérica de uma grandeza y
  em função de uma grandeza x.
  '''
  ds = []

  for i in range(len(x)):
    if i == 0: # se eu estou na borda da esquerda
      d = (y[i+1] - y[i])/(x[i+1] - x[i]) 
    elif i == len(x)-1: # borda da direita
      d = (y[i] - y[i-1])/(x[i] - x[i-1]) 
    else: 
      d = (y[i+1] - y[i-1])/(x[i+1] - x[i-1])
    ds.append(d)
  return ds

In [ ]:
derivada_energia = derivada(EE[0], Ts)

In [ ]:
plt.plot(Ts, derivada_energia)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

tc = 2*J/(k*np.log(1+np.sqrt(2)))
ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
ax.plot(Ts, derivada_energia, '-', label=f'h = {0}')
ax.set_xlabel('T[K]', fontsize=18)
ax.set_ylabel(r'$\frac{d<E>}{dT}$', fontsize=18)
ax.set_title('Derivada da Energia Média Função da Temperatura', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.axvline(x = find_max_point(derivada_energia, Ts), color='red',linestyle='dashed', label=f'$T_c$ = {find_max_point(derivada_energia, Ts):.3f}')
ax.legend(fontsize=(14))


In [ ]:
def find_max_point(y,x):
  '''
  Essa função encontra o x referente ao máximo de um conjunto de dados
  '''
  aux = 0
  auxx = [] # variáveis auxiliares
  for i in range(np.size(y)):
    if y[i] > y[aux]:
      aux = i
      auxx = []
    elif y[i] == aux:
      auxx.append(i)
  if len(auxx)>1:
    t = []
    for i in auxx:
      t.append(x[i])
    return np.mean(t)
  else:
    return x[aux]

In [ ]:
#print(Ts)
#print(derivada_energia)
find_max_point(derivada_energia,Ts)

In [ ]:
def run_grid_simulation(Nmax, T, h, J, k, N, M):
  config0 = initial_config(N,M) # cria condição inicial aleatória
  plot_config(config0, 'config0') # Plota e salva a configuração inicial
  beta = 1/(k*T)
  E = calculate_energy(config0, J, h)
  for cont in range(Nmax): # itero nas varreduras
    E = sweep_config(config0, J, h, beta, E) 
  plot_config(config0, f'config_final_{T}') # ploto e salvo a configuracao final

In [ ]:
#run_grid_simulation(2000,1,0,1,1,200,200)

In [ ]:
def run_simulation_alt(Nmax, Ts, h, J, k, Ns, inicio_aleatorio = True):

  '''
  Aqui, efetivamente, rodo a simulação (mas para diferentes valores de N)
  '''

  EE = [] # lista para salvar os dados de energia para cada h
  C = [] # lista para salvar os dados de capacidade térmica
  MM = [] # lista para salvar os dados de magnetizacao para cada h
  CHI = [] # lista para salvar os dados de susceptibilidade magnetica para cada h

  for nn in range(np.size(Ns)): # para cada h na lista hs
    Es = [] # lista para guardar as energias médias
    Es2 = [] # lista para guardar as energias medias quadradas
    Mags = [] # lista para guardar as magnetizacoes médias
    Mags2 = [] # lista para guardar as magnetizacoes médias quadradas
    Cs = [] # ista para guardar as capacidades térmicas
    Chis = [] # lista para guardar as susceptibilidades magnéticas
    if inicio_aleatorio:
      config0 = np.copy(initial_config(Ns[nn], Ns[nn])) # crio uma configuração aleatória de spins
    else:
      config0 = np.ones((Ns[nn],Ns[nn]))
    for t in range(np.size(Ts)):  # para cada temperatura na lista ts
      beta = 1/(k*Ts[t]) # calculo beta
      ms = [] # lista para guardar as magnetizacoes por sitio (em módulo)
      es = [] # lista para guardar as energias
      spins_config = np.copy(config0) 
      E = calculate_energy(spins_config, J, h) # calculo a energia inicial
      m = np.absolute(np.mean(spins_config)) # calculo a magnetizacao inicial
      es.append(E) # salvo a primeira energia
      ms.append(m) # salvo a primeira magnet
      for cont in range(Nmax-1): # itero nas varreduras
        E = sweep_config(spins_config, J, h, beta, E) # varro e salvo a energia nova
        es.append(E) # salvo a energia
        ms.append(np.sum(spins_config)) # salvo a magnetizacao
      Mags.append(np.mean(ms)) # salvo a magnetizacao média total
      Mags2.append(np.mean(np.array(ms)**2))
      Es.append(np.mean(es)) # salvo a energia média total
      Es2.append(np.mean(np.array(es)**2))
      Cs.append(-(Es[-1]**2 - Es2[-1])/(k**2*Ts[t]**2)) # Salvo o calor específico
      Chis.append(-(Mags[-1]**2 - Mags2[-1])/(k*Ts[t])) # Salvo a susceptibilidade magnética
    EE.append(np.array(Es)/(Ns[nn]**2))
    MM.append(np.absolute(np.array(Mags))/(Ns[nn]**2))
    C.append(np.array(Cs)/(Ns[nn]**2))
    CHI.append(np.array(Chis)/(Ns[nn]**2)) 
  EE = np.array(EE)
  MM = np.array(np.absolute(MM))
  C = np.array(C)
  CHI = np.array(CHI)
  return Ts, Ns, EE, MM, C,CHI

In [ ]:
def plot_mag_en_alt(Ts, Ns, EE, MM, C, CHI, k, J, lit = True):
  fig, ax = plt.subplots(figsize=(8,8))

  tc = 2*J/(k*np.log(1+np.sqrt(2)))
  t_aux = np.arange(0.1,4,0.01)
  m_onsager = np.array([mag_onsager(k,J,t) for t in t_aux])
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for nn in range(np.size(Ns)):
      ax.plot(Ts, MM[nn], 'o-', label=f'N = {Ns[nn]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel('|<M(T)>|', fontsize=18)
      ax.legend(fontsize=(14))
      ax.set_title('Magnetização Média em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  if lit:
    ax.plot(t_aux, m_onsager, 'k-', label=f'Onsager Solution')
  plt.savefig(f'mag_vs_temp_ns')
  
  fig1, ax = plt.subplots(figsize=(8,8))
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for nn in range(np.size(Ns)):
      ax.plot(Ts, EE[nn], 's-', label=f'N = {Ns[nn]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel('E', fontsize=18)
      ax.legend(fontsize=(14))
      ax.set_title('Energia Média em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  plt.savefig(f'en_vs_temp_ns')

  fig2, ax = plt.subplots(figsize=(8,8))
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for nn in range(np.size(Ns)):
      ax.plot(Ts, C[nn], 's-', label=f'N = {Ns[nn]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel('C(T)', fontsize=18)
      ax.legend(fontsize=(14))
      ax.set_title('Capacidade Térmica em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  plt.savefig(f'C_vs_temp_hs')

  fig3, ax = plt.subplots(figsize=(8,8))
  ax.axvline(x = tc, color='black',linestyle='dashed', label=r'$T_c \approx 2.27$')
  for nn in range(np.size(Ns)):
      ax.plot(Ts, CHI[nn], 's-', label=f'N = {Ns[nn]}')
      ax.set_xlabel('T[K]', fontsize=18)
      ax.set_ylabel(r'$\chi$', fontsize=18)
      ax.legend(fontsize=(14))
      ax.set_title('Susceptibilidade Magnética em Função da Temperatura', fontsize=18)

      ax.tick_params(axis='both', which='major', labelsize=14)
  plt.savefig(f'CHI_vs_temp_hs')


In [ ]:
Ts_alt, Ns_alt, EE_alt, MM_alt, C_alt, CHI_alt = run_simulation_alt(1000, np.arange(0.1,4.1,0.1), 0, 1, 1, [5,7], False)

In [ ]:
plot_mag_en_alt(Ts_alt, Ns_alt, EE_alt, MM_alt, C_alt, CHI_alt, 1, 1)